## Refinamento e recuperação das imagens dos Posts de boletins

#### Importação das dependencias

In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

#### Definindo Constantes

In [2]:
CHROME_PATH = '/usr/bin/google-chrome'
CHROMEDRIVER_PATH = '../bin/chromedriver'
WINDOW_SIZE = "1920,1080"

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH

#### Lendo DataSet de Posts

In [3]:
df_posts = pd.read_csv("../dataset/posts_facebook.csv")
df_posts.head()

,post_id,source,date,text,image,url
0,962170830896016,Prefeitura de Icém,"[datetime.datetime(2020, 6, 29, 17, 0, 11)]",🚨Confira o Boletim Informativo🚨 de Icém refere...,https://scontent.fcgh7-1.fna.fbcdn.net/v/t1.0-...,/story.php?story_fbid=962170830896016&id=23449...
1,961495100963589,Prefeitura de Icém,"[datetime.datetime(2020, 6, 28, 17, 0, 15)]",🚨Confira o Boletim Informativo🚨 de Icém refere...,https://scontent.fcgh7-1.fna.fbcdn.net/v/t1.0-...,/story.php?story_fbid=961495100963589&id=23449...
2,961187350994364,Prefeitura de Icém,"[datetime.datetime(2020, 6, 28, 10, 15, 59)]",Quero agradecer a cada um doador de sangue mui...,https://scontent.fcgh7-1.fna.fbcdn.net/v/t1.0-...,/story.php?story_fbid=961187350994364&id=23449...
3,960726124373820,Prefeitura de Icém,"[datetime.datetime(2020, 6, 27, 17, 32, 15)]",🚨Confira o Boletim Informativo🚨 de Icém refere...,https://scontent.fcgh7-1.fna.fbcdn.net/v/t1.0-...,/story.php?story_fbid=960726124373820&id=23449...
4,960053934441039,Prefeitura de Icém,"[datetime.datetime(2020, 6, 26, 19, 31, 11)]",Entrega dos cadernos de atividades complementa...,https://scontent.fcgh7-1.fna.fbcdn.net/v/t1.0-...,/story.php?story_fbid=960053934441039&id=23449...


In [5]:
df_posts = df_posts.fillna("")
posts_boletins = df_posts[df_posts['text'].str.contains("Boletim")]

In [10]:
def get_images(url):
    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options
    )
    print(f"GET {url}")
    driver.get(url)
    
    time.sleep(3)
    element = driver.find_element_by_id("expanding_cta_close_button")
    element.click()

    time.sleep(3)
    element = driver.find_element_by_xpath('//a[contains(@href,"photo")]')
    element.click()

    time.sleep(5)
    photo = driver.find_element_by_xpath("//img[@class='spotlight']")
    src = photo.get_attribute("src")
    driver.quit()
    return src

#### Optendo imagens pelo WebBrowser

In [11]:
imagens = []
for _, post in posts_boletins.iterrows():
    print(f"Processando {post.post_id}")
    try:
        imagens.append(
            get_images(f"https://www.facebook.com{post.url}")
        )
    except NoSuchElementException:
        print(f"Erro em -> https://www.facebook.com{post.url}")
    
    

Processando 962170830896016


/home/cesaraugusto/.pyenv/versions/3.7.7/envs/scrap2/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


GET https://www.facebook.com/story.php?story_fbid=962170830896016&id=234499770329796
Processando 961495100963589
GET https://www.facebook.com/story.php?story_fbid=961495100963589&id=234499770329796
Processando 960726124373820
GET https://www.facebook.com/story.php?story_fbid=960726124373820&id=234499770329796
Processando 959994087780357
GET https://www.facebook.com/story.php?story_fbid=959994087780357&id=234499770329796
Processando 959277597852006
GET https://www.facebook.com/story.php?story_fbid=959277597852006&id=234499770329796
Processando 957834814662951
GET https://www.facebook.com/story.php?story_fbid=957834814662951&id=234499770329796
Processando 957074851405614
GET https://www.facebook.com/story.php?story_fbid=957074851405614&id=234499770329796
Processando 956420174804415
GET https://www.facebook.com/story.php?story_fbid=956420174804415&id=234499770329796
Processando 955575804888852
GET https://www.facebook.com/story.php?story_fbid=955575804888852&id=234499770329796
Processando

#### Salvando dataset refinado

In [16]:
posts_boletins['image'] = imagens

/home/cesaraugusto/.pyenv/versions/3.7.7/envs/scrap2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
posts_boletins.to_csv('../dataset/posts_boletins_refine.csv')